In [1]:
# Setup
import sys
import numpy as np
import tensorflow as tf
import Utils.PokemonUtils as pkmnUtils
from matplotlib import pyplot as plt
from Models import baseline
import importlib

utils = pkmnUtils.PokemonUtils(rel_loc="")
utils.loadAllGIFs()
x_train, x_val, x_test, y_train, y_val, y_test = utils.generateXYSplitsV1()

USE_GPU = True
if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

def trainBatches(batch_size):
    for batch in range(int(len(x_train) / batch_size)):
        yield x_train[batch * batch_size: (batch + 1) * batch_size], y_train[batch * batch_size: (batch + 1) * batch_size]

def valBatches(batch_size):
    for batch in range(int(len(x_val) / batch_size)):
        yield x_val[batch * batch_size: (batch + 1) * batch_size], y_val[batch * batch_size: (batch + 1) * batch_size]

def testBatches(batch_size):
    for batch in range(int(len(x_test) / batch_size)):
        yield x_test[batch * batch_size: (batch + 1) * batch_size], y_test[batch * batch_size: (batch + 1) * batch_size]

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/benjaminshare/pokemon-learning
25 done!
50 done!
75 done!
100 done!
125 done!
150 done!
175 done!
200 done!
225 done!
250 done!
275 done!
300 done!
325 done!
350 done!
375 done!
400 done!
425 done!
450 done!
475 done!
500 done!
525 done!
550 done!
575 done!
600 done!
625 done!
650 done!
675 done!
700 done!
725 done!
750 done!
775 done!
800 done!
825 done!
850 done!
875 done!
900 done!
925 done!
all 940 done!
smallest: [14 15 29  4]
largest: [299 188 234   4]
x_train: (601, 128, 128, 4)
x_val:   (151, 128, 128, 4)
x_test:  (188, 128, 128, 4)
y_train: (601,)
y_val:   (151,)
y_test:  (188,)


In [ ]:
# Run on PokemonUtils update
import Utils.PokemonUtils as pkmnUtils
importlib.reload(pkmnUtils)  # update import if PokemonUtils.py changes
utils = pkmnUtils.PokemonUtils(utils=utils)

In [26]:
# Run on model update
from Models import baseline
importlib.reload(baseline)

<module 'Models.baseline' from '/home/benjaminshare/pokemon-learning/Models/baseline.py'>

In [2]:
def check_accuracy(sess, dset, x, scores):
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        feed_dict = {x: x_batch, is_training: 0}
        scores_np = sess.run(scores, feed_dict=feed_dict)
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

In [29]:
# Train model
model = baseline.ShallowNet()
num_epochs = 2
learning_rate = 1e-3
batch_size = 32
print_every = 10

tf.reset_default_graph()    
with tf.device(device):
    x = tf.placeholder(tf.float32, [None, 128, 128, 4])
    y = tf.placeholder(tf.int32, [None])

    is_training = tf.placeholder(tf.bool, name='is_training')

    scores = model.model_init_fn(x)

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
    loss = tf.reduce_mean(loss)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in range(num_epochs):
            print('Starting epoch %d' % epoch)
            for x_batch, y_batch in trainBatches(batch_size):
                feed_dict = {x: x_batch, y: y_batch, is_training:1}
                loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
                if t % print_every == 0:
                    print('Iteration %d, loss = %.4f' % (t, loss_np))
                    check_accuracy(sess, valBatches(batch_size), x, scores)
                    print()
                t += 1
                    

Starting epoch 0
Iteration 0, loss = 27.4148
Got 6 / 128 correct (4.69%)

Iteration 10, loss = 2.9800
Got 12 / 128 correct (9.38%)

Starting epoch 1
Iteration 20, loss = 2.8188
Got 12 / 128 correct (9.38%)

Iteration 30, loss = 2.8024
Got 11 / 128 correct (8.59%)



In [ ]:
# Test model
acc = None
pass
print(acc)